In [ ]:
import pandas as pd
import numpy as np
print("Bibliothèques importées avec succès")

In [ ]:
# Chargement du dataset
netflix = pd.read_csv('../data/netflix_titles.csv')

In [ ]:
# print(f"Les dimensions du tableau sont : {netflix.shape}")
netflix.info()
netflix.head()

In [ ]:
# Conversion de 'date_added' (objet) en 'date_added_feature' (datetime)
netflix['date_added_feature'] = netflix['date_added'].str.strip() # Elimination des espaces blancs en debut et en fin
netflix['date_added_feature'] = pd.to_datetime(netflix['date_added_feature'], errors='coerce') # Permet de forcer la conversion en réelle date exploitable

# creation de la colonne 'year_added' a partir de 'date_added_feature' : Extraction de l'année d'ajout 
netflix['year_added'] = netflix['date_added_feature'].dt.year

# creation de la colonne 'month_added' a partir de 'date_added_feature' : Extraction du mois d'ajout 
netflix['month_added'] = netflix['date_added_feature'].dt.month

# creation de la colonne 'added_day_of_week' a partir de 'date_added_feature' : Extraction du jour d'ajout 
netflix['added_day_of_week'] = netflix['date_added_feature'].dt.day

# Creation de la colonne 'lag_time' : qui est la durée entre l'ajout sur netflix et la sortie
netflix['lag_time'] = netflix['year_added'] - netflix['release_year']

netflix.head()


In [ ]:
# Séparation de la durée des films et des series
'''Initialisation des colonnes a remplir'''
netflix['duration_min'] = np.nan # permet de creer un colonne pour la duree des film
netflix['duration_seasons'] = np.nan # permet de creer une colonne pour le nombre de saison des series

'''Création des masques servant a departager les films et series'''
mask_films = ((netflix['type']=='Movie') & (netflix['duration'].notna())) # Masque pour les films
mask_series = ((netflix['type']=='TV Show') & (netflix['duration'].notna())) # Masque pour les series

'''Application des masques et séparation des films et series'''
# .loc[masque_films, 'colonne_à_remplir'] = ...
netflix.loc[mask_films, 'duration_min'] = netflix.loc[mask_films, 'duration'] # Application du masque film sur la colonne et extraction de la duree des film
netflix['duration_min'] = netflix['duration_min'].str.replace(' min', '').astype(float) # Conversion de la duree qui en 'str' en 'float'


netflix.loc[mask_series, 'duration_seasons'] = netflix.loc[mask_series, 'duration'] # Application du masque series sur la colonne et extraction de la duree des series
netflix['duration_seasons'] = netflix['duration_seasons'].str.replace(' Seasons', '').str.replace(' Season', '').astype(float) # Conversion de la duree qui en 'str' en 'float'

netflix.head()

In [ ]:
# Extraction de la categorie principal et pays de chaque film et series
'''
En partant du principe que le premier pays listé est le pays principal
Pareil pour la catégorie, en partant du principe que la première catégorie de la liste est la catégorie principal
'''

# Pour les pays
# .str.split(',') : Coupe la chaîne à chaque virgule (renvoie une liste)
# .str[0]         : Sélectionne le premier élément de cette liste
netflix['main_country'] = netflix['country'].str.split(',').str[0]

# Pour les catégories
netflix['main_genre'] = netflix['listed_in'].str.split(',').str[0]

netflix.head()

In [ ]:
# Ma liste de colonnes final
columns_final = [
    'show_id', 
    'type', 
    'title', 
    'main_country', 
    'main_genre', 
    'release_year', 
    'date_added_feature',
    'year_added', 
    'month_added', 
    'added_day_of_week', 
    'lag_time', 
    'duration_min', 
    'duration_seasons'
]

# Nouveau dataframe :
netflix_cleaned = netflix[columns_final].copy()
netflix_cleaned.head()

In [ ]:
# Telechargement du nouveau dataset nettoyé
netflix_cleaned.to_csv("netflix_cleaned.csv", index=False)
print("DataFrame nettoyé sauvegardé dans netflix_cleaned.csv !")